In [55]:
# if a module import does not work, use below
%pip install requests
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
     --------------------------------------- 11.6/11.6 MB 54.7 MB/s eta 0:00:00
     ------------------------------------- 346.6/346.6 KB 22.4 MB/s eta 0:00:00
     --------------------------------------- 15.9/15.9 MB 54.5 MB/s eta 0:00:00
     ------------------------------------- 508.0/508.0 KB 31.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


# CSE587 Project

Saagnik Sarbadhikari, 50592327

Marcus Hartman, 50398874

Bharath Reddy, 50563984

# Forecasting the Rental Housing Economy of Various Areas Using Machine Learning

Data Retrieval

In [56]:
import json
import requests
import pandas

In [ ]:
# saag

In [58]:
# Marcus' data

# huduser.gov
huduser_key = open('tokens/huduser').readline()

huduser_MN = requests.get("https://www.huduser.gov/hudapi/public/fmr/statedata/MN", headers={"Authorization": f"Bearer {huduser_key}"}).json()

huduser_TX = requests.get("https://www.huduser.gov/hudapi/public/fmr/statedata/TX", headers={"Authorization": f"Bearer {huduser_key}"}).json()


# zillow.com
zillow_rentals = pandas.read_csv("https://files.zillowstatic.com/research/public_csvs/zori/Metro_zori_uc_sfrcondomfr_sm_month.csv?t=1728278819")
# this is home values forecasts, not rentals. but still poses significance 
zillow_forecasts = pandas.read_csv("https://files.zillowstatic.com/research/public_csvs/zhvf_growth/Metro_zhvf_growth_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1728278819")


# rentcast.io
# NOTE: any request using the API key will accumulate towards the monthly limit (31). please do not go over this

# the commented out part uses the API key you put into tokens/rentcast (req. credit card info). 
# the service is free but still requires credit card info-- if you find something better use it. update latex and make a little note next to it and we're all good
# rentcast_key = open('tokens/rentcast').readline()

# rentcast_TX = requests.get("https://api.rentcast.io/v1/listings/rental/long-term?state=TX", headers={"accept": "application/json", "X-Api-Key": rentcast_key})
# f = open("results/rentcast_TX", "w")
# f.write(json.dumps(rentcast_TX.json()))
# f.close()

# rentcast_MN = requests.get("https://api.rentcast.io/v1/listings/rental/long-term?state=MN", headers={"accept": "application/json", "X-Api-Key": rentcast_key})
# f = open("results/rentcast_MN", "w")
# f.write(json.dumps(rentcast_MN.json()))
# f.close()

# loads data stored from api calls above. costs money if we go above 31 calls / month
rentcast_TX = json.loads(open('results/rentcast_TX').readline())
rentcast_MN = json.loads(open('results/rentcast_MN').readline())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664 entries, 0 to 663
Columns: 121 entries, RegionID to 2024-08-31
dtypes: float64(116), int64(2), object(3)
memory usage: 627.8+ KB


In [ ]:
# bharath

Data Cleaning

In [ ]:
# need 10 different cleaning operations for all of our sources


EDA

In [ ]:
# saag

In [ ]:
# marcus

In [ ]:
# bharath